# PSからPLを制御するためのプログラム

In [ ]:
import numpy as np

In [5]:
N = 10              # 最大文字数

kmj_data = ['(　́ω`)ノ']

## 前処理

### 使用できる文字のリストを読み込み

In [3]:
char_list = []
with open('char_list.txt' , 'r', encoding='utf-8') as file:
  for line in file:
    char_list.append(line.replace('\n', ''))

### 文字を整数値に変換

In [6]:
# 文字リストの番号を顔文字中の番号を対応付け
kmj_index = []
for kmj in kmj_data:
  kmj = list(kmj)
  kmj += ['<PAD>' for _ in range(N - len(kmj))]
  temp = []
  for c in kmj:
    try:
      temp.append(char_list.index(c))
    except:
      temp.append(char_list.index('<UNK>'))
  kmj_index.append(temp)

In [7]:
kmj_index

[[2, 7, 26, 8, 52, 5, 33, 0, 0, 0]]

## FPGAとの通信

In [99]:
from pynq import PL
PL.reset()

### FPGA書き込み

In [100]:
from pynq import Overlay

ol = Overlay("kmj_gen.bit")

### IPコア定義

In [101]:
dma = ol.axi_dma_0
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

In [102]:
from pynq import MMIO
top_mmio = MMIO(ol.ip_dict['top_0']['phys_addr'], ol.ip_dict['top_0']['addr_range'])

### AXI Stream でデータを書き込み

In [103]:
from pynq import allocate

data_size = N
input_buffer = allocate(shape=(data_size,), dtype=np.uint8)
output_buffer = allocate(shape=(data_size,), dtype=np.uint8)

In [104]:
for i in range(data_size):
    input_buffer[i] = kmj_index[0][i]

In [105]:
input_buffer, output_buffer

(PynqBuffer([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [117]:
# データ送信
dma_send.transfer(input_buffer)

### AXI LITE で回路全体を制御

In [118]:
top_mmio.write(0, 0)

In [ ]:
# slv_reg1[1:0] : mode
top_mmio.write(4, 0b00)

In [119]:
# slv_reg0[2:0] : {set, run, rst_n}
top_mmio.write(0, 3)

In [120]:
# slv_reg2[0] : finish
top_mmio.read(8)

1

### AXI Stream でデータを読み込み

In [110]:
dma_recv.transfer(output_buffer)

In [111]:
output_buffer

PynqBuffer([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

## 整数列を文字列に変換

In [ ]:
def convert_str(x):
  x = np.array(kg.char_list)[x]
  x = [c for c in x if c not in ['<PAD>', '<UNK>']]

  return ''.join(x)

In [ ]:
print(convert_str(output_buffer))